In [1]:
#set up autoreload
%load_ext autoreload
%autoreload 2

import polars as pl
from pathlib import Path

import importlib  
fa_etl = importlib.import_module("fa-etl")

In [9]:
county = "36061"
path_to_dir = "/Users/claireboyd/internships/mansueto/firstamerican-etl/"
output_filepath_ranked = path_to_dir+f"dev/{county}/staging/ranked_ValHist{county}.parquet"

taxhist = pl.read_parquet(path_to_dir+f"dev/{county}/staging/TaxHist{county}.parquet")
valhist = pl.read_parquet(path_to_dir+f"dev/{county}/staging/ValHist{county}.parquet")
annual = pl.read_parquet(path_to_dir+f"dev/{county}/staging/Prop{county}.parquet")
sales_ranked = pl.read_parquet(path_to_dir+f"dev/{county}/staging/ranked_Deed{county}.parquet")
sales = pl.read_parquet(path_to_dir+f"dev/{county}/staging/Deed{county}.parquet")

appr = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/appr.parquet", low_memory=True)
assd = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/assd.parquet", low_memory=True)
market = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/market.parquet", low_memory=True)

In [10]:
assd.join(
    other=market,
    how="left",
    on=['PropertyID', 'Year'],
).sink_parquet(Path(output_filepath_ranked), compression="snappy")

In [12]:
ranked_valhist = pl.read_parquet(path_to_dir+f"dev/{county}/staging/ranked_ValHist{county}.parquet")

In [22]:
fa_etl.is_lazydataframe_empty(path_to_dir+f"dev/{county}/staging/ranked_ValHist{county}.parquet")

False

In [23]:
ranked_valhist.shape

(3566623, 4)

In [8]:
valhist.schema

OrderedDict([('PropertyID', Int64),
             ('AssdTotalValue', Int64),
             ('AssdYear', Int64),
             ('MarketTotalValue', Int64),
             ('MarketValueYear', Int64),
             ('ApprTotalValue', Int64),
             ('ApprYear', Int64),
             ('TaxableYear', Int64)])

In [24]:
merged = pl.read_parquet(path_to_dir+f"dev/{county}/unified/merged.parquet")

In [27]:
merged.head()

PropertyID,Year,Value,MarketTotalValue,ApprTotalValue,SitusLatitude,SitusLongitude,SitusFullStreetAddress,SitusCity,SitusState,SitusZIP5,FIPS,SitusCensusTract,SitusCensusBlock,SaleAmt,TaxAmt,TaxAmtAdjusted
i64,i64,i64,i64,null,f64,f64,str,str,str,str,str,str,str,i64,i64,f64
91847870,2018,117677,261504,null,40.750965,-73.982198,"""425 5TH AVE AP…","""NEW YORK""","""NY""","""10016""","""36061""","""008200""","""2004""",1580000,1310424,13104.24
91849008,2018,247162,1462000,null,40.750447,-73.997723,"""362 W 30TH ST""","""NEW YORK""","""NY""","""10001""","""36061""","""009700""","""4000""",4400000,3114052,31140.52
91848046,2018,71155,158123,null,40.738643,-73.987954,"""254 PARK AVE S…","""NEW YORK""","""NY""","""10010""","""36061""","""005200""","""1000""",712775,773352,7733.52
91846549,2018,149310,331800,null,40.729262,-74.004785,"""63 DOWNING ST …","""NEW YORK""","""NY""","""10014""","""36061""","""006700""","""2002""",2475000,1527372,15273.72
91847537,2018,128574,285721,null,40.73889,-73.987904,"""260 PARK AVE S…","""NEW YORK""","""NY""","""10010""","""36061""","""005200""","""1000""",1625000,1496148,14961.48
